In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
from loguru import logger

import gptchem
from gptchem.tuner import Tuner
from gptchem.formatter import RegressionFormatter
from gptchem.querier import Querier
from gptchem.extractor import RegressionExtractor

with open("qm9_key_smiles_0_val_u0_atom_b3lyp.pickle", "rb") as handle:
        qm9_data = pickle.load(handle)

def split_dictionary(input_dict, chunk_size):
    res = []
    new_dict = {}
    for k, v in input_dict.items():
        if len(new_dict) < chunk_size:
            new_dict[k] = v
        else:
            res.append(new_dict)
            new_dict = {k: v}
    res.append(new_dict)
    return res

/u/g/m/gmerz2/miniconda3/envs/chemGPT/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for i, mydict in split_dictionary(input_dict, chunk_size):
    qm9_data_pd = pd.DataFrame(list(qm9_data.items()), columns=["SMILES", "B3LYP atomization energy in kcal/mol"])

    # encode the data into prompts and completions
    formatter = RegressionFormatter(representation_column='SMILES',
        label_column='B3LYP atomization energy in kcal/mol',
        property_name='atomization energy in kcal/mol',
        num_digits=4
        )

    formatted_data = formatter.format_many(qm9_data_pd)

    # split the data into training and test set
    train, test = train_test_split(formatted_data, test_size=0.1, random_state=42)

    # enable gptchem logging
    logger.enable("gptchem")

    # define the logging level
    LEVEL = "DEBUG"
    logger.add("my_log_file.log", level=LEVEL, enqueue=True)

    # initialize the tuner
    tuner = Tuner()
    tune_summary = tuner(train)

    print(tune_summary)

    # initialize the querier
    querier = Querier('ada') # use the model called 'ada'

    # get the completions (assuming the test frame we created above)
    completions = querier(test)

    # extract the predictions
    extractor = RegressionExtractor()
    predictions = extractor(completions)
    print(predictions)
    metrics = gptchem.evaluator.get_regression_metrics(predictions, test['label'])
    print(metrics)

KeyboardInterrupt: 